In [2]:
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd

# 讀取中和區範圍的shp
county_data = gpd.read_file('./圖像資料/鄉(鎮、市、區)界線(TWD97經緯度)1140318/TOWN_MOI_1140318.shp')
Zhonghe_District_data = county_data[county_data['TOWNNAME'] == '新北市中和區']

# 將房屋轉成GeoDataFrame
df = pd.read_csv('./data/real_estate_cleaned.csv', encoding = 'utf-8')

gdf_real_estate = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df['Lng'], df['Lat']), crs = 'EPSG: 4326')
gdf_real_estate = gdf_real_estate.to_crs(Zhonghe_District_data.crs)

# 轉成平面座標系
gdf_real_estate = gdf_real_estate.to_crs(epsg = 3826)

df_mrt = pd.read_csv('./data/northern-taiwan-mrt-station.csv', encoding = 'utf-8')
gdf_mrt = gpd.GeoDataFrame(df_mrt, geometry = gpd.points_from_xy(df_mrt['lon'], df_mrt['lat']), crs = 'EPSG: 4326')
gdf_mrt_Zhonghe1 = gdf_mrt[gdf_mrt['address'].str.contains('新北市中和區', na = False)]

# 去除'景安'站的重複值
gdf_mrt_Zhonghe = gdf_mrt_Zhonghe1.drop_duplicates(subset = ['station_name_tw'])
gdf_mrt_Zhonghe = gdf_mrt_Zhonghe.to_crs(epsg = 3826)
print(gdf_mrt_Zhonghe)

# 設定捷運站緩衝區
gdf_mrt_Zhonghe['buffer_500m'] = gdf_mrt_Zhonghe.buffer(500)
print(gdf_mrt_Zhonghe)


# 建立站點房價列表
result_list = []

for idx, station in gdf_mrt_Zhonghe.iterrows():
    nearby = gdf_real_estate[gdf_real_estate.geometry.within(station['buffer_500m'])]
    mean_price = nearby['單價元平方公尺'].mean()
    result_list.append({'站名':station['station_name_tw'], '平均房價': mean_price, '房屋數量': len(nearby)})
df_station_prices = pd.DataFrame(result_list)
print(result_list)

print(df_station_prices)

# df_station_prices.to_csv('./data/房價統計表.csv', index = 0, header = 1, encoding = 'utf-8')


# 設定房屋緩衝區
gdf_real_estate['buffer_500m'] = gdf_real_estate.buffer(500)
print(gdf_real_estate)

# 建立站點數量列表
result_list2 = []

for idx, house in gdf_real_estate.iterrows():
    nearby = gdf_mrt_Zhonghe[gdf_mrt_Zhonghe.geometry.within(house['buffer_500m'])]
    result_list2.append({'地址': house['土地區段位置建物區段門牌'], '站點數量': len(nearby)})
df_house_mrts = pd.DataFrame(result_list2)
print(df_house_mrts)
within_500m = df_house_mrts.groupby('站點數量')
print(within_500m.size())

# 增加一欄標註房屋離哪個捷運站最近(尚未成功)
'''
from dotenv import load_dotenv
load_dotenv()
import os
load_key = os.environ.get('GOOGLE_MAP_API_Key')

import googlemaps
import time

gmaps = googlemaps.Client(key = load_key)
Address = df['土地區段位置建物區段門牌'].to_list()
address = Address

ids = []
for addr in address:
    geocode_result = gmaps.geocode(addr)
    loc = geocode_result[0]['geometry']['location']
    time.sleep(0.1)
    place_result = gmaps.places_nearby(loc, keyword = '捷運', type = 'subway_station', radius = 500)
    for place in place_result['results']:
        ids.append(place['name'])

mrt_station_name = pd.DataFrame(ids)

print(ids)
print(mrt_station_name)
'''


    station_code construction_id station_name_tw station_name_en line_code  \
72           O01             O19             南勢角      Nanshijiao         O   
73           O02             O18              景安          Jingan         O   
74           O03             O17            永安市場   Yongan Market         O   
123          Y09              Y8             秀朗橋  Xiulang Bridge         Y   
124          Y10              Y9              景平        Jingping         Y   
126          Y12             Y11              中和         Zhonghe         Y   
127          Y13             Y12              橋和          Qiaohe         Y   
128          Y14             Y13              中原       Zhongyuan         Y   

    line_name          address        lat         lon  \
72      中和新蘆線      新北市中和區捷運路6號  24.990045  121.509237   
73      中和新蘆線    新北市中和區景平路486號  24.993905  121.505113   
74      中和新蘆線    新北市中和區中和路388號  25.002876  121.511231   
123       環狀線      新北市中和區景平路2號  24.990549  121.525051   
124       環狀

In [1]:
import folium
import json
import pandas as pd
import geopandas as gpd
from folium.plugins import MarkerCluster
import math

df_mrt = pd.read_csv('./data/northern-taiwan-mrt-station.csv', encoding = 'utf-8')
print(df_mrt)

df_mrt_Zhonghe = df_mrt[df_mrt['address'].str.contains('新北市中和區', na = False)]
print(df_mrt_Zhonghe)

# 畫出捷運站Circle圖
Zhonghe_map = folium.Map(location = [24.9972, 121.5042], tiles = 'openstreetmap', zoom_start = 15)
for row in range(0, len(df_mrt_Zhonghe)):
    folium.Circle(
        location = [df_mrt_Zhonghe.iloc[row]['lat'], df_mrt_Zhonghe.iloc[row]['lon']],
        popup = df_mrt_Zhonghe.iloc[row]['station_name_tw'],
        radius = 500,
        color = '#69b3a2',
        fill = True,
        fill_color = '#69b3a2').add_to(Zhonghe_map)
Zhonghe_map

# 再疊加上房屋MarkerCluster圖
marker_cluster = MarkerCluster()
Zhonghe_real_estate = pd.read_csv('./data/real_estate_cleaned.csv', encoding = 'utf-8')
print(Zhonghe_real_estate)

for idx, row in Zhonghe_real_estate.iterrows():
    if not math.isnan(row['Lng']) and not math.isnan(row['Lat']):
        marker_cluster.add_child(folium.Marker([row['Lat'], row['Lng']], popup = "{}，每平方公尺單價{}元".format(row['土地區段位置建物區段門牌'], row['單價元平方公尺'])))
        
Zhonghe_map.add_child(marker_cluster)

# 將地圖存成html
Zhonghe_map.save('./data/MRT_Zhonghe.html')

    station_code construction_id station_name_tw      station_name_en  \
0           BR01            BR13             動物園           Taipei Zoo   
1           BR02            BR12              木柵                Muzha   
2           BR03            BR11            萬芳社區    Wanfang Community   
3           BR04            BR10            萬芳醫院     Wanfang Hospital   
4           BR05             BR9              辛亥               Xinhai   
..           ...             ...             ...                  ...   
151          A17             A17              領航             Linghang   
152          A18             A18           高鐵桃園站  Taoyuan HSR Station   
153          A19             A19          桃園體育園區  Taoyuan Sports Park   
154          A20             A20              興南              Xingnan   
155          A21             A21              環北              Huanbei   

    line_code line_name           address        lat         lon  
0          BR       文湖線    台北市文山區新光路二段32號  24.998197  12

In [2]:
import folium
import pandas
from folium.plugins import HeatMap

Zhonghe_map = folium.Map(location = [24.9972, 121.5042], tiles = 'openstreetmap', zoom_start = 15)

HeatMap(data = Zhonghe_real_estate[['Lat', 'Lng']], radius = 30).add_to(Zhonghe_map)
Zhonghe_map

# 將熱力圖存成html
Zhonghe_map.save('./data/Zhonghe_real_estate_HeatMap.html')